Taken from Georg's parallelproj

In [ ]:
import array_api_compat.cupy as xp

import parallelproj
from array_api_compat import to_device
import array_api_compat.numpy as np
import matplotlib.pyplot as plt

if "numpy" in xp.__name__:
    dev = "cpu"
elif "cupy" in xp.__name__:
    dev = xp.cuda.Device(0)
elif "torch" in xp.__name__:
    dev = "cuda"

num_rings = 32
scanner = parallelproj.RegularPolygonPETScannerGeometry(
    xp,
    dev,
    radius=70.0,
    num_sides=12,
    num_lor_endpoints_per_side=30,
    lor_spacing=1.2,
    ring_positions=xp.linspace(-48, 48, num_rings),
    symmetry_axis=1, 
)

lor_desc = parallelproj.RegularPolygonPETLORDescriptor(
    scanner,
    radial_trim=10,
    max_ring_difference=8,
    sinogram_order=parallelproj.SinogramSpatialAxisOrder.RVP,
)

proj = parallelproj.RegularPolygonPETProjector(
    lor_desc, img_shape=(128, 24, 128), voxel_size=(.75, 3.0, .75)
)

fig = plt.figure(figsize=(8, 8))
ax1 = fig.add_subplot(111, projection="3d")
proj.show_geometry(ax1)
fig.tight_layout()
fig.show()


In [ ]:

test = np.load('Brainweb.npz')
vol = test['PET'][0, 19:109, 100:-100, 100:-100]
vol = vol[:, 8:-8, 8:-8]
vol = vol[28:52, :, :]
vol = np.swapaxes(vol, 0, 1)

import matplotlib.pyplot as plt
from ipywidgets import interact
def show_slices(slice_idx):
    plt.imshow(vol[:,slice_idx,:], cmap='gray')
    plt.colorbar()
    plt.show()
    
interact(show_slices, slice_idx=(0, vol.shape[1]-1, 1))

In [ ]:
xstart, xend = proj._lor_descriptor.get_lor_coordinates(views=proj._views)

x = xp.array(vol)

x_fwd = parallelproj.joseph3d_fwd(
                xstart, xend, x, proj._img_origin, proj._voxel_size
            )

x_back = parallelproj.joseph3d_back(
                xstart, xend, proj._img_shape, proj._img_origin, proj._voxel_size, x_fwd,
            )

dict_data = {}

dict_data['xstart'] = xstart.get()
dict_data['xend'] = xend.get()
dict_data['img_origin'] = proj._img_origin.get()
dict_data['voxel_size'] = proj._voxel_size.get()
dict_data['img_shape'] = proj._img_shape
dict_data['x'] = x.get()
dict_data['meas'] = x_fwd.get() + 100.

np.save("large_regular_polygon_geom.npy", dict_data)

import matplotlib.pyplot as plt
from ipywidgets import interact
def show_slices(slice_idx):
    plt.imshow(x_back.get()[:,slice_idx,:], cmap='gray')
    plt.colorbar()
    plt.show()
    
interact(show_slices, slice_idx=(0, vol.shape[1]-1, 1))